#### **Парсинг: подготовка данных для модели**

Весь парсер сделан на [yargy](https://github.com/natasha/yargy), подсветка реализована с помощью [ipymarkup](https://github.com/natasha/ipymarkup).
Все инструменты из open source проекта `natasha`  

In [1]:
import dataframe as df
import parser as ps

path = 'data/train.json'

In [2]:
data = df.reads_json(path) 
data.head()

,text,label,start,stop,search
id,,,,,
809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,1279,1343,Размер обеспечения исполнения контракта 6593.2...
854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,1222,1318,Поставщик должен предоставить обеспечение испо...
4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,1297,1343,Размер обеспечения исполнения контракта 10.00%
184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,1304,1350,Размер обеспечения исполнения контракта 10.00%
211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,1302,1348,Размер обеспечения исполнения контракта 10.00%


##### **false negative/positive errors**

Определяем записи по `extracted_part` с ответами и без

In [11]:
len = data['stop'] - data['start']

answ_on = data[len != 0].index
answ_off = data[len == 0].index

Определяем ошибки 1 и 2 рода, где парсер нашел пункт, хотя ответа нет и где не нашел, но ответ есть

В идеале оба массива ошибок должны быть пустыми, но в приоритете избавиться от ошибок типа *false negative* (II), так как мы сразу отбросим эти записи и больше не сможем с ними работать

Ошибки типа *false positive* (I) могут отсеяться на следующих этапах парсинга

In [5]:
label_off = []   #   search all labeloff docs

for i, _ in data.iterrows():    
    marks = ps.get_marks(
        _['text'], _['label'])
    if not marks:
        label_off.append(i)       

label_on = [x for x in data.index if x not in label_off]          

In [6]:
errorsI = [x for x in label_on if x in answ_off]
errorsII = [x for x in label_off if x in answ_on] 


print(errorsI, errorsII)

[476192036, 588528614, 833988154, 219363951, 471097685, 14015810] []


Корректируем правила для парсера:    [parser/rules/label.py](parser/rules/label.py)

```c
CONTRACT = rule(
    or_(
        normalized('обеспечение'),
        normalized('исполнение'),
    ).repeatable(),
    normalized('настоящий').optional(),
    or_(
        normalized('контракт'),
        normalized('договор'),
    )
)
```

In [13]:
#      >>>execute 4th, 5th code cells

In [7]:
data = data.loc[label_on]
data.shape

(1498, 5)

##### **NER: поиск дополнительных сущностей**

In [3]:
import dataframe as df
import parser as ps

path = 'data/train.json'

data = df.reads_json(path) 

label_off = [21387522, 988512459, 576814118, 33321982, 870221477, 841199412, 659636986, 530135745, 103381269, 774423675, 767518830, 84333860, 842259306, 568278777, 398016768, 247509154, 186227613, 742758410, 41858643, 279678657, 641564028, 899079794, 883593150, 566870066, 218092753, 633942681, 348860902, 504104789, 310575279, 416901600, 480448230, 379814163, 722054566, 321458781, 898595603, 464383825, 195226664, 780443903, 589056981, 410875162, 374366795, 247159152, 89682092, 657384927, 425520652, 159399266, 482313075, 894571448, 67082098, 254389104, 902797643, 123213155, 793891511, 988693551, 490969848, 574894248, 58876711, 481449707, 931962258, 394540200, 462277146, 384224571, 695164638, 908753490, 977032556, 323658138, 933595516, 383798572, 586645301, 30076414, 966458400, 816725356, 163655004, 359802858, 689018053, 877239444, 175884542, 487971169, 895055033, 701794530, 314181537, 457049634, 301667980, 102249797, 390988811, 203664165, 214648417, 150243287, 85524565, 633704262, 327661343, 884320964, 175966212, 325780003, 252638411, 95089965, 499694856, 491030460, 705282233, 978164618, 908780351, 777830838, 62494133, 184429686, 960701326, 328251163, 376171914, 287889263, 758238915, 800036488, 808739861, 589396852, 821740539, 683436950, 889631488, 449432350, 421175213, 750578324, 853848068, 558092963, 503906217, 640524438, 573061698, 521050542, 94961499, 860644451, 154786355, 59733154, 445125138, 74345292, 191213064, 353223249, 558319334, 568949681, 710378801, 897474405, 228483166, 521227314, 937713785, 576330245, 383357205, 954175818, 168223198, 648724951, 472239980, 744167309, 856330320, 604894431, 61237725, 825877886, 814561214, 114758421, 885471240, 753877092, 728877463, 564086098, 556024184, 880506700, 689412582, 423787183, 484115071, 747317433, 189772990, 155043411, 262719817, 784758104, 693419707, 484398444, 259008463, 975974006, 688576534, 27864617, 966996823, 338987269, 598266674, 561237319, 227123674, 370408517, 158397066, 546365142, 580361221, 496636639, 503051655, 705839048, 201509913, 928033358, 37186938, 427883942, 892721516, 843472863, 314228254, 654942446, 79764055, 266572819, 696588941, 485503185, 841385708, 240757843, 971288916, 496154367, 926782072, 918234141, 30093679, 56336706, 866396752, 100721352, 695326447, 259352116, 541345914, 533170647, 830754475, 297024744, 465552654, 700398177, 53349068, 163876804, 762481292, 785021421, 888990974, 719126687, 32394515, 592144870, 303984403, 706280324, 415524602, 932088635, 236293110, 944535517, 880802714, 605248511, 528966580, 961697129, 615066621, 686005102, 477741501, 682728869, 799538669, 263036851, 303693485, 720376558, 909772075, 673444485, 677793788, 911698012, 871916176, 660980540, 800859874, 360944883, 416279128, 917231113, 387377068, 998879542, 555083461, 685751564, 291713866, 281148396, 265383438, 848887402, 968130729, 711430708, 920883070, 312286816, 110092843, 958537814, 340078714, 540120178, 643589590, 940359335, 855412825, 903923266, 765979602, 945573559, 460300374, 571309346, 160915063, 125157783, 14785464, 256399317, 408331151, 673034280, 576628577, 977930644, 250923192, 613094776, 828659340, 390815281, 300790544, 289894159, 522101245, 627854560, 588286734, 800430498, 285239088, 686209257, 360349411, 396622857, 58251022, 214609521, 94811748, 903507299, 926039184]

label_on = [x for x in data.index if x not in label_off]   
data = data.loc[label_on]

In [4]:
for i, _ in data.iloc[:3].iterrows():    
    
    lb = _['label']
    txt = _['text']
    _off = ps.get_offset(txt, lb)

    txt = txt[_off:]
    marks = ps.find_marks(txt, lb,
                        amount=True,
                        additional=True)
    off_ = marks[-1].stop
    txt = txt[:off_]
    
    ps.show_markup(txt,marks)   #   show marks in first doc

In [ ]:
amount_off = []

for i, _ in data.iterrows():    
    
    lb = _['label']
    txt = _['text']

    marks = ps.get_marks(txt, lb,
                        amount=True)

    if not (ps.get_entry(marks)):
        amount_off.append(i)
        ps.show_markup(txt,marks)

In [10]:
for i, _ in data.loc[not_found_p].iterrows():    
    
    lb = _['label']
    txt = _['text']
    # _off = ps.find_offset(txt, lb)

    # txt = txt[_off:]
    marks = ps.get_marks(txt, lb,
                        amount=True,
                        additional=True)
    # off_ = marks[-1].stop
    # txt = txt[:off_]
    ps.show_markup(txt,marks)

In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    MoneyExtractor,
    Doc

    
)


from razdel import tokenize
from razdel import sentenize

tokens = list(tokenize(data[0]['text']))
for _ in tokens:
    print(i)

    
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)


# text = text[spans[0].start:spans[-1].stop]
# text = 'Способ обеспечения исполнения контракта, гарантийных обязательств'
# doc = Doc(text)

# doc.segment(segmenter)
# doc.tag_morph(morph_tagger)
# doc.parse_syntax(syntax_parser)

# sent = doc.sents[0]
# # sent.morph.print()


# sent.syntax.print()
